
# Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation

This is an example of applying :class:`~sklearn.decomposition.NMF` and
:class:`~sklearn.decomposition.LatentDirichletAllocation` on a corpus
of documents and extract additive models of the topic structure of the
corpus.  The output is a plot of topics, each represented as bar plot
using top few words based on weights.

Non-negative Matrix Factorization is applied with two different objective
functions: the Frobenius norm, and the generalized Kullback-Leibler divergence.
The latter is equivalent to Probabilistic Latent Semantic Indexing.

The default parameters (n_samples / n_features / n_components) should make
the example runnable in a couple of tens of seconds. You can try to
increase the dimensions of the problem, but be aware that the time
complexity is polynomial in NMF. In LDA, the time complexity is
proportional to (n_samples * iterations).


In [ ]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from time import time

import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
import numpy as np

In [ ]:
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20
batch_size = 128
init = "nndsvda"


def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
data = pd.read_csv('./uiuc_top_all.csv')
# data_controversial = pd.read_csv('./uiuc_controversial_all.csv')
# data = pd.concat([data, data_controversial])

# replace NaN with empty string or 0 for numerical columns
data['PostText'] = data['PostText'].replace(np.nan, '', regex=True)
data['Title'] = data['Title'].replace(np.nan, '', regex=True)
data['Sentiment'] = data['Sentiment'].replace(np.nan, 0, regex=True)

years = [str(i) for i in range(2008, 2024)]

data_by_year = {year: data[data['Date'].str.contains(year)] for year in years}

# remove years with less than 2 posts
data_by_year = {year: data_by_year[year] for year in years if len(data_by_year[year]) > 10}
years = list(data_by_year.keys())

print(years)
data

In [ ]:
year_model = {}
for year in years:
    df = data_by_year[year]

    # concat post titles to body text (data)
    post_titles = df['Title'].to_list()
    post_body = df['PostText'].to_list()

    dataset = [str(title + ' ' + body) for title, body in zip(post_titles, post_body)]

    data_samples = dataset[:n_samples]
    # print([type(d) for d in data_samples])


    # Use tf-idf features for NMF.
    print("Extracting tf-idf features for NMF...")
    tfidf_vectorizer = TfidfVectorizer(
        max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
    )
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data_samples)
    print("done in %0.3fs." % (time() - t0))

    # Fit the NMF model
    print(
        "\n" * 2,
        "Fitting the NMF model (generalized Kullback-Leibler "
        "divergence) with tf-idf features, n_samples=%d and n_features=%d..."
        % (n_samples, n_features),
    )
    t0 = time()
    nmf = NMF(
        n_components=n_components,
        random_state=1,
        init=init,
        beta_loss="kullback-leibler",
        solver="mu",
        max_iter=1000,
        alpha_W=0.00005,
        alpha_H=0.00005,
        l1_ratio=0.5,
    ).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
    # plot_top_words(
    #     nmf,
    #     tfidf_feature_names,
    #     n_top_words,
    #     "Topics in NMF model (generalized Kullback-Leibler divergence)",
    # )
    year_model[year] = (nmf, tfidf_feature_names)


In [99]:
import json
topics = []
posts_by_topic = {}
for year, (nmf, tfidf_feature_names) in year_model.items():
    # print(f"Year: {year}")
    for topic_idx, topic in enumerate(nmf.components_):
        top_features_ind = topic.argsort()[-3:]
        top_features = tfidf_feature_names[top_features_ind]
        # print(f"Topic {topic_idx + 1}: {', '.join(top_features)}")

        # Get the posts in the year's data that include the top features
        posts_with_top_features = data_by_year[year][data_by_year[year]['PostText'].str.contains('|'.join(top_features))]

        posts_by_topic[", ".join(top_features)] = [
           { "title": title, "url": url } 
           for title, url in zip(posts_with_top_features['Title'], posts_with_top_features['URL'])]

        proportion_posts = len(posts_with_top_features)

        # Calculate the sentiment score for each post and sum them
        sentiment_score = sum(posts_with_top_features['Sentiment'])

        # print(f"Sentiment Score: {sentiment_score}")

        topics.append(
            {
                "year": int(year),
                "words": ", ".join(top_features),
                "sentiment": sentiment_score,
                "count": proportion_posts,
            }
        )

normalized_topics = []
for year in years:
    year_topics = [t for t in topics if t['year'] == int(year)]
    ranking = np.argsort([t['count'] for t in year_topics])
    for i, t in enumerate(year_topics):
        t['rank'] = int(ranking[i])
        normalized_topics.append(t)

with open('topics.json', 'w') as f:
  json.dump(normalized_topics, f)

with open('posts_by_topic.json', 'w') as f:
  json.dump(posts_by_topic, f)

TypeError: Object of type zip is not JSON serializable

In [ ]:
# # Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# # to filter out useless terms early on: the posts are stripped of headers,
# # footers and quoted replies, and common English words, words occurring in
# # only one document or in at least 95% of the documents are removed.

# print("Loading dataset...")
# t0 = time()
# data, _ = fetch_20newsgroups(
#     shuffle=True,
#     random_state=1,
#     remove=("headers", "footers", "quotes"),
#     return_X_y=True,
# )
# data_samples = data[:n_samples]
# print("done in %0.3fs." % (time() - t0))

In [ ]:


# # Use tf-idf features for NMF.
# print("Extracting tf-idf features for NMF...")
# tfidf_vectorizer = TfidfVectorizer(
#     max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
# )
# t0 = time()
# tfidf = tfidf_vectorizer.fit_transform(data_samples)
# print("done in %0.3fs." % (time() - t0))

# # Use tf (raw term count) features for LDA.
# print("Extracting tf features for LDA...")
# tf_vectorizer = CountVectorizer(
#     max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
# )
# t0 = time()
# tf = tf_vectorizer.fit_transform(data_samples)
# print("done in %0.3fs." % (time() - t0))
# print()

# # Fit the NMF model
# print(
#     "Fitting the NMF model (Frobenius norm) with tf-idf features, "
#     "n_samples=%d and n_features=%d..." % (n_samples, n_features)
# )
# t0 = time()
# nmf = NMF(
#     n_components=n_components,
#     random_state=1,
#     init=init,
#     beta_loss="frobenius",
#     alpha_W=0.00005,
#     alpha_H=0.00005,
#     l1_ratio=1,
# ).fit(tfidf)
# print("done in %0.3fs." % (time() - t0))


# tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
# plot_top_words(
#     nmf, tfidf_feature_names, n_top_words, "Topics in NMF model (Frobenius norm)"
# )

# # Fit the NMF model
# print(
#     "\n" * 2,
#     "Fitting the NMF model (generalized Kullback-Leibler "
#     "divergence) with tf-idf features, n_samples=%d and n_features=%d..."
#     % (n_samples, n_features),
# )
# t0 = time()
# nmf = NMF(
#     n_components=n_components,
#     random_state=1,
#     init=init,
#     beta_loss="kullback-leibler",
#     solver="mu",
#     max_iter=1000,
#     alpha_W=0.00005,
#     alpha_H=0.00005,
#     l1_ratio=0.5,
# ).fit(tfidf)
# print("done in %0.3fs." % (time() - t0))

# tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
# plot_top_words(
#     nmf,
#     tfidf_feature_names,
#     n_top_words,
#     "Topics in NMF model (generalized Kullback-Leibler divergence)",
# )

# # Fit the MiniBatchNMF model
# print(
#     "\n" * 2,
#     "Fitting the MiniBatchNMF model (Frobenius norm) with tf-idf "
#     "features, n_samples=%d and n_features=%d, batch_size=%d..."
#     % (n_samples, n_features, batch_size),
# )
# t0 = time()
# mbnmf = MiniBatchNMF(
#     n_components=n_components,
#     random_state=1,
#     batch_size=batch_size,
#     init=init,
#     beta_loss="frobenius",
#     alpha_W=0.00005,
#     alpha_H=0.00005,
#     l1_ratio=0.5,
# ).fit(tfidf)
# print("done in %0.3fs." % (time() - t0))


# tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
# plot_top_words(
#     mbnmf,
#     tfidf_feature_names,
#     n_top_words,
#     "Topics in MiniBatchNMF model (Frobenius norm)",
# )

# # Fit the MiniBatchNMF model
# print(
#     "\n" * 2,
#     "Fitting the MiniBatchNMF model (generalized Kullback-Leibler "
#     "divergence) with tf-idf features, n_samples=%d and n_features=%d, "
#     "batch_size=%d..." % (n_samples, n_features, batch_size),
# )
# t0 = time()
# mbnmf = MiniBatchNMF(
#     n_components=n_components,
#     random_state=1,
#     batch_size=batch_size,
#     init=init,
#     beta_loss="kullback-leibler",
#     alpha_W=0.00005,
#     alpha_H=0.00005,
#     l1_ratio=0.5,
# ).fit(tfidf)
# print("done in %0.3fs." % (time() - t0))

# tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
# plot_top_words(
#     mbnmf,
#     tfidf_feature_names,
#     n_top_words,
#     "Topics in MiniBatchNMF model (generalized Kullback-Leibler divergence)",
# )

# print(
#     "\n" * 2,
#     "Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
#     % (n_samples, n_features),
# )
# lda = LatentDirichletAllocation(
#     n_components=n_components,
#     max_iter=5,
#     learning_method="online",
#     learning_offset=50.0,
#     random_state=0,
# )
# t0 = time()
# lda.fit(tf)
# print("done in %0.3fs." % (time() - t0))

# tf_feature_names = tf_vectorizer.get_feature_names_out()
# plot_top_words(lda, tf_feature_names, n_top_words, "Topics in LDA model")

In [98]:
import colorsys

def adjust_colors(colors, saturation_diff):
    adjusted_colors = []
    for color in colors:
        # Convert hex color to RGB
        rgb = tuple(int(color[i:i+2], 16) for i in (1, 3, 5))

        # Convert RGB to HSV
        hsv = colorsys.rgb_to_hsv(rgb[0]/255, rgb[1]/255, rgb[2]/255)

        # Adjust luminance
        adjusted_saturation = max(0, min(1, hsv[1] + saturation_diff))

        # Convert HSV back to RGB
        adjusted_rgb = colorsys.hsv_to_rgb(hsv[0], hsv[1], adjusted_saturation)

        # Convert RGB to hex
        adjusted_hex = '#%02x%02x%02x' % tuple(int(c * 255) for c in adjusted_rgb)

        adjusted_colors.append(adjusted_hex)

    return adjusted_colors

# Example usage
colors = [
    "#67001f",
    "#6a011f",
    "#6d0220",
    "#700320",
    "#730421",
    "#760521",
    "#790622",
    "#7b0722",
    "#7e0823",
    "#810923",
    "#840a24",
    "#870b24",
    "#8a0c25",
    "#8c0d26",
    "#8f0f26",
    "#921027",
    "#941127",
    "#971228",
    "#9a1429",
    "#9c1529",
    "#9f172a",
    "#a1182b",
    "#a41a2c",
    "#a61c2d",
    "#a81d2d",
    "#aa1f2e",
    "#ad212f",
    "#af2330",
    "#b12531",
    "#b32732",
    "#b52933",
    "#b72b34",
    "#b82e35",
    "#ba3036",
    "#bc3238",
    "#be3539",
    "#bf373a",
    "#c13a3b",
    "#c33c3d",
    "#c43f3e",
    "#c6413f",
    "#c74441",
    "#c94742",
    "#ca4943",
    "#cc4c45",
    "#cd4f46",
    "#ce5248",
    "#d0544a",
    "#d1574b",
    "#d25a4d",
    "#d45d4e",
    "#d56050",
    "#d66252",
    "#d86554",
    "#d96855",
    "#da6b57",
    "#db6d59",
    "#dd705b",
    "#de735d",
    "#df755f",
    "#e07861",
    "#e17b63",
    "#e27d65",
    "#e48067",
    "#e58369",
    "#e6856b",
    "#e7886d",
    "#e88b6f",
    "#e98d71",
    "#ea9073",
    "#eb9276",
    "#ec9578",
    "#ed977a",
    "#ee9a7c",
    "#ee9c7f",
    "#ef9f81",
    "#f0a183",
    "#f1a486",
    "#f2a688",
    "#f2a88b",
    "#f3ab8d",
    "#f4ad90",
    "#f4af92",
    "#f5b295",
    "#f5b497",
    "#f6b69a",
    "#f6b89c",
    "#f7ba9f",
    "#f7bda1",
    "#f8bfa4",
    "#f8c1a6",
    "#f8c3a9",
    "#f9c5ab",
    "#f9c7ae",
    "#f9c9b0",
    "#facab3",
    "#faccb5",
    "#faceb8",
    "#fad0ba",
    "#fad2bc",
    "#fad3bf",
    "#fad5c1",
    "#fbd7c4",
    "#fbd8c6",
    "#fbdac8",
    "#fbdbca",
    "#fbddcc",
    "#fadecf",
    "#fae0d1",
    "#fae1d3",
    "#fae2d5",
    "#fae3d7",
    "#fae5d8",
    "#fae6da",
    "#f9e7dc",
    "#f9e8de",
    "#f9e9e0",
    "#f8eae1",
    "#f8eae3",
    "#f7ebe4",
    "#f7ece6",
    "#f6ede7",
    "#f6ede8",
    "#f5eee9",
    "#f4eeeb",
    "#f4efec",
    "#f3efed",
    "#f2efed",
    "#f1efee",
    "#f0f0ef",
    "#eff0f0",
    "#eef0f0",
    "#edf0f1",
    "#eceff1",
    "#ebeff1",
    "#eaeff2",
    "#e9eff2",
    "#e7eef2",
    "#e6eef2",
    "#e5edf2",
    "#e3edf2",
    "#e2ecf2",
    "#e0ecf2",
    "#dfebf2",
    "#ddeaf2",
    "#dbeaf1",
    "#dae9f1",
    "#d8e8f1",
    "#d6e7f0",
    "#d4e6f0",
    "#d3e6f0",
    "#d1e5ef",
    "#cfe4ef",
    "#cde3ee",
    "#cbe2ee",
    "#c9e1ed",
    "#c7e0ed",
    "#c5dfec",
    "#c2ddec",
    "#c0dceb",
    "#bedbea",
    "#bcdaea",
    "#bad9e9",
    "#b7d8e8",
    "#b5d7e8",
    "#b2d5e7",
    "#b0d4e6",
    "#aed3e6",
    "#abd1e5",
    "#a9d0e4",
    "#a6cfe3",
    "#a3cde3",
    "#a1cce2",
    "#9ecae1",
    "#9cc9e0",
    "#99c7e0",
    "#96c6df",
    "#93c4de",
    "#91c3dd",
    "#8ec1dc",
    "#8bc0db",
    "#88beda",
    "#85bcd9",
    "#83bbd8",
    "#80b9d7",
    "#7db7d7",
    "#7ab5d6",
    "#77b3d5",
    "#74b2d4",
    "#71b0d3",
    "#6faed2",
    "#6cacd1",
    "#69aad0",
    "#66a8cf",
    "#64a7ce",
    "#61a5cd",
    "#5ea3cc",
    "#5ba1cb",
    "#599fca",
    "#569dc9",
    "#549bc8",
    "#5199c7",
    "#4f98c6",
    "#4d96c5",
    "#4b94c4",
    "#4892c3",
    "#4690c2",
    "#448ec1",
    "#428cc0",
    "#408bbf",
    "#3e89be",
    "#3d87bd",
    "#3b85bc",
    "#3983bb",
    "#3781ba",
    "#3680b9",
    "#347eb7",
    "#337cb6",
    "#317ab5",
    "#3078b4",
    "#2e76b2",
    "#2d75b1",
    "#2c73b0",
    "#2a71ae",
    "#296fad",
    "#286dab",
    "#266baa",
    "#2569a8",
    "#2467a6",
    "#2365a4",
    "#2164a2",
    "#2062a0",
    "#1f609e",
    "#1e5e9c",
    "#1d5c9a",
    "#1b5a98",
    "#1a5895",
    "#195693",
    "#185490",
    "#17528e",
    "#164f8b",
    "#154d89",
    "#134b86",
    "#124983",
    "#114781",
    "#10457e",
    "#0f437b",
    "#0e4178",
    "#0d3f75",
    "#0c3d73",
    "#0a3b70",
    "#09386d",
    "#08366a",
    "#073467",
    "#063264",
    "#053061",
]
satruationdiff = 0.1
adjusted_colors = adjust_colors(colors, satruationdiff)
print(adjusted_colors)

['#ff004c', '#ff024a', '#ff044a', '#ff0648', '#ff0849', '#ff0a47', '#ff0c47', '#ff0e46', '#ff1046', '#ff1145', '#ff1345', '#ff1443', '#ff1644', '#ff1745', '#ff1a43', '#ff1b44', '#ff1d43', '#ff1e43', '#ff2143', '#ff2243', '#ff2443', '#ff2644', '#ff2844', '#ff2b45', '#ff2c44', '#ff2e45', '#ff3045', '#ff3245', '#ff3546', '#ff3747', '#ff3947', '#ff3b48', '#ff3f49', '#ff414a', '#ff434b', '#ff474c', '#ff494d', '#ff4c4d', '#ff4e4f', '#ff5150', '#ff5351', '#ff5753', '#ff5a53', '#ff5c54', '#ff5f56', '#ff6257', '#ff6559', '#ff665a', '#ff6a5b', '#ff6d5d', '#ff6f5d', '#ff725f', '#ff7461', '#ff7763', '#ff7a63', '#ff7d65', '#fd7e66', '#fb7f67', '#fa8168', '#f88269', '#f6846a', '#f4856b', '#f3866c', '#f1876d', '#f0896e', '#ee896e', '#ec8b6f', '#ea8c70', '#e98d71', '#e78e71', '#e48e72', '#e38f73', '#e18f74', '#e09174', '#dc9075', '#db9176', '#d99276', '#d79277', '#d59278', '#d29278', '#d19379', '#ce9279', '#cc927a', '#c9927a', '#c7927b', '#c5927b', '#c3927b', '#c0917c', '#be917c', '#bc917c', '#ba907c'